# Image summary and visual question answering

This notebooks shows some preliminary work on Image Captioning and Visual question answering with lavis. It is mainly meant to explore its capabilities and to decide on future research directions. We package our code into a `ammico` package that is imported here:

In [1]:
from ammico import utils as mutils
from ammico import display as mdisplay
import ammico.summary as sm

Set an image path as input file path.

In [2]:
images = mutils.find_files(
    path="data/",
    limit=10,
)

In [3]:
mydict = mutils.initialize_dict(images)

## Create captions for images and directly write to csv

Here you can choose between two models: "base" or "large"

In [4]:
obj = sm.SummaryDetector(mydict)
summary_model, summary_vis_processors = obj.load_model("base")
# summary_model, summary_vis_processors = obj.load_model("large")

  0%|          | 0.00/2.50G [00:00<?, ?B/s]

  0%|          | 4.37M/2.50G [00:00<00:58, 45.8MB/s]

  1%|          | 23.8M/2.50G [00:00<00:19, 139MB/s] 

  2%|▏         | 43.0M/2.50G [00:00<00:15, 167MB/s]

  2%|▏         | 64.0M/2.50G [00:00<00:14, 183MB/s]

...

 97%|█████████▋| 2.42G/2.50G [00:14<00:00, 163MB/s]

 98%|█████████▊| 2.44G/2.50G [00:14<00:00, 187MB/s]

 98%|█████████▊| 2.46G/2.50G [00:15<00:00, 118MB/s]

 99%|█████████▉| 2.48G/2.50G [00:15<00:00, 144MB/s]

100%|██████████| 2.50G/2.50G [00:15<00:00, 177MB/s]

In [5]:
for key in mydict:
    mydict[key] = sm.SummaryDetector(mydict[key]).analyse_image(
        summary_model, summary_vis_processors
    )

Convert the dictionary of dictionaries into a dictionary with lists:

In [6]:
outdict = mutils.append_data_to_dict(mydict)
df = mutils.dump_df(outdict)

Check the dataframe:

In [7]:
df.head(10)

,filename,const_image_summary,3_non-deterministic summary
0,data/102730_eng.png,two people in blue coats spray disinfection a van,[two people wearing blue uniforms spraying out...
1,data/102141_2_eng.png,"a collage of images including a corona sign, a...","[many pictures include people, signs, and a fe..."
2,data/106349S_por.png,a man wearing a face mask while looking at a c...,"[a man in a face mask talking into the radio, ..."


Write the csv file:

In [8]:
df.to_csv("./data_out.csv")

## Manually inspect the summaries

To check the analysis, you can inspect the analyzed elements here. Loading the results takes a moment, so please be patient. If you are sure of what you are doing.

`const_image_summary` - the permanent summarys, which does not change from run to run (analyse_image).

`3_non-deterministic summary` - 3 different summarys examples that change from run to run (analyse_image). 

In [9]:
mdisplay.explore_analysis(mydict, identify="summary")

AttributeError: module 'ammico.display' has no attribute 'explore_analysis'

## Generate answers to free-form questions about images written in natural language. 

Set the list of questions

In [10]:
list_of_questions = [
    "How many persons on the picture?",
    "Are there any politicians in the picture?",
    "Does the picture show something from medicine?",
]

In [11]:
for key in mydict:
    mydict[key] = sm.SummaryDetector(mydict[key]).analyse_questions(list_of_questions)

  0%|          | 0.00/1.35G [00:00<?, ?B/s]

  0%|          | 6.01M/1.35G [00:00<00:25, 55.4MB/s]

  1%|          | 12.0M/1.35G [00:00<00:30, 46.6MB/s]

  1%|▏         | 17.5M/1.35G [00:00<00:31, 44.6MB/s]

  2%|▏         | 24.6M/1.35G [00:00<00:27, 51.2MB/s]

...

 98%|█████████▊| 1.32G/1.35G [00:20<00:00, 47.3MB/s]

 99%|█████████▊| 1.33G/1.35G [00:21<00:00, 55.0MB/s]

 99%|█████████▉| 1.34G/1.35G [00:21<00:00, 57.6MB/s]

100%|█████████▉| 1.34G/1.35G [00:21<00:00, 48.2MB/s]

100%|██████████| 1.35G/1.35G [00:21<00:00, 67.4MB/s]

In [12]:
mdisplay.explore_analysis(mydict, identify="summary")

AttributeError: module 'ammico.display' has no attribute 'explore_analysis'

Convert the dictionary of dictionarys into a dictionary with lists:

In [13]:
outdict2 = mutils.append_data_to_dict(mydict)
df2 = mutils.dump_df(outdict2)

In [14]:
df2.head(10)

,filename,const_image_summary,3_non-deterministic summary,How many persons on the picture?,Are there any politicians in the picture?,Does the picture show something from medicine?
0,data/102730_eng.png,two people in blue coats spray disinfection a van,[two people wearing blue uniforms spraying out...,2,no,yes
1,data/102141_2_eng.png,"a collage of images including a corona sign, a...","[many pictures include people, signs, and a fe...",1,no,yes
2,data/106349S_por.png,a man wearing a face mask while looking at a c...,"[a man in a face mask talking into the radio, ...",1,yes,yes


In [15]:
df2.to_csv("./data_out2.csv")